# Sección 1: Análisis de Datos


#### Import library

In [132]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Parte A: Análisis de un conjunto de datos

### DATA LOADING

In [133]:
df = pd.read_csv('customer_data.csv', sep = ',', na_values='#N/D')

In [134]:
df.head()

,CustomerID,Age,Location,TotalPurchases,LastPurchaseDate,LifetimeValue,Contacted,PurchaseProbability
0,1,62,West,1,4/3/2024 20:45,1898.15,Yes,0.49
1,2,65,South,5,45:50.9,5172.70,Yes,0.98
2,3,18,South,4,5/25/2024 20:45,5846.43,Yes,0.24
3,4,21,South,14,45:50.9,7896.67,No,0.37
4,5,21,North,12,45:50.9,6104.11,Yes,0.05


### BUSINESS ANALYTICS

In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CustomerID           100 non-null    int64  
 1   Age                  100 non-null    int64  
 2   Location             100 non-null    object 
 3   TotalPurchases       100 non-null    int64  
 4   LastPurchaseDate     100 non-null    object 
 5   LifetimeValue        100 non-null    float64
 6   Contacted            100 non-null    object 
 7   PurchaseProbability  100 non-null    float64
dtypes: float64(2), int64(3), object(3)
memory usage: 6.4+ KB


In [136]:
age_average = df['Age'].mean() 
location_frequent = df['Location'].mode()[0]
frequency_average_purchase = df['TotalPurchases'].mean()

average_profile = {
    'Edad Promedio': age_average,
    'Ubicación Más Frecuente': location_frequent,
    'Frecuencia de Compra Promedio': frequency_average_purchase
}

average_profile


{'Edad Promedio': 41.57,
 'Ubicación Más Frecuente': 'East',
 'Frecuencia de Compra Promedio': 9.48}

Conclusiones: 

- `Edad Promedio: 42%,`
- `Ubicación Más Frecuente: 'East',`
- `Frecuencia de Compra Promedio: 9%`

### Identificar los 10 clientes con el mayor valor de vida (Lifetime Value)

In [148]:
top_10_lifetime_value = df.nlargest(10, 'LifetimeValue')[['CustomerID', 'LifetimeValue']]

top_10_lifetime_value

# Los siguientes CustomerID conrrespoenden a los 10 clientes con el mayor valor de vida 

,CustomerID,LifetimeValue
19,20,9851.82
28,29,9808.93
16,17,9745.16
40,41,9533.42
38,39,9269.51
75,76,9259.12
96,97,9114.39
72,73,8931.29
44,45,8840.31
79,80,8838.82


### Encuentra las tres variables que más influyen en la probabilidad de que un cliente haga una compra.

 Cuantificar los valores de `location` para analizar si se encuentra dentro las tres varialbes que influyan en la probabilidad de compra

In [138]:
location_mapping = {'West': 1, 'South': 2, 'North': 3, 'East': 4}
df['Location_Quantified'] = df['Location'].map(location_mapping)

df

,CustomerID,Age,Location,TotalPurchases,LastPurchaseDate,LifetimeValue,Contacted,PurchaseProbability,Location_Quantified
0,1,62,West,1,4/3/2024 20:45,1898.15,Yes,0.49,1
1,2,65,South,5,45:50.9,5172.70,Yes,0.98,2
2,3,18,South,4,5/25/2024 20:45,5846.43,Yes,0.24,2
3,4,21,South,14,45:50.9,7896.67,No,0.37,2
4,5,21,North,12,45:50.9,6104.11,Yes,0.05,3
...,...,...,...,...,...,...,...,...,...
95,96,32,North,6,45:50.9,1013.38,No,0.39,3
96,97,33,West,13,45:50.9,9114.39,No,0.53,1
97,98,38,North,13,45:50.9,921.42,No,0.94,3
98,99,53,South,3,45:50.9,8023.75,No,0.89,2


In [139]:
# Seleccionar solo las columnas numericas
df_numerical = df.select_dtypes(include=['number'])

In [140]:
correlations = df_numerical.corr()['PurchaseProbability'].drop('PurchaseProbability').abs()

# Seleccionar las tres variables con mayor correlación
influential_variables = correlations.nlargest(3)

influential_variables 

Age                    0.224970
LifetimeValue          0.219919
Location_Quantified    0.108288
Name: PurchaseProbability, dtype: float64

### Perfil promedio  
- Calcular la edad promedio 
- ubicación más frecuente 
- frecuencia de compra promedio

Conclusiones:

las tres variables que más influyen en la probabilidad de que un cliente haga una compra son 
- `age with 22%`
- `LifetimeValue with 22%`
- `Location with  10%`

## Parte B: Generación de Estrategias de Marcación 

### Describa una estrategia para priorizar las llamadas a clientes, incluyendo cualquier segmentación de clientes que considere relevante

### Estrategia para Priorizar las Llamadas a Clientes

- Segmentación por Valor de Vida `LifetimeValue`: Priorizar clientes con un alto valor de vida

In [141]:
## Segmentación por Valor de Vida (Lifetime Value)
top_10_lifetime_value

,CustomerID,LifetimeValue
19,20,9851.82
28,29,9808.93
16,17,9745.16
40,41,9533.42
38,39,9269.51
75,76,9259.12
96,97,9114.39
72,73,8931.29
44,45,8840.31
79,80,8838.82


- Segmentación por Probabilidad de Compra `PurchaseProbability`: Utilizar la probabilidad de compra como indicador para centrar los esfuerzos en clientes más propensos a realizar compras.


In [142]:
top_10_Purchase_Probability = df.nlargest(10, 'PurchaseProbability')[['CustomerID', 'PurchaseProbability']]

## Segmentación por Probabilidad de Compra (PurchaseProbability)
top_10_Purchase_Probability


,CustomerID,PurchaseProbability
1,2,0.98
72,73,0.98
77,78,0.98
23,24,0.97
79,80,0.97
88,89,0.97
69,70,0.96
28,29,0.94
97,98,0.94
16,17,0.93


### ¿Qué métricas utilizarías para evaluar el éxito de esta estrategia de marcación?

In [143]:
# Suponiendo que ya hemos realizado las llamadas y ahora queremos evaluar el éxito

# Métricas
conversion_rate = len(df[df['Contacted'] == 'Yes']) / len(df)  # Tasa de Conversión
total_revenue = df[df['Contacted'] == 'Yes']['LifetimeValue'].sum()  # Ingresos Generados
total_cost = 2000000  # Supongamos que tienes el costo total de las llamadas
roi = (total_revenue - total_cost) / total_cost  # Ratio de Retorno de Inversión

# Retención de Clientes (ejemplo de cálculo)
clientes_retencionados = df[(df['Contacted'] == 'Yes') & (df['TotalPurchases'] > 0)]
retencion_rate = len(clientes_retencionados) / len(df[df['Contacted'] == 'Yes'])

# Mostrar métricas
print(f"\nTasa de Conversión: {conversion_rate}")
print(f"Ingresos Generados: {total_revenue}")
print(f"Ratio de Retorno de Inversión (ROI): {roi}")
print(f"Porcentaje de Retención de Clientes: {retencion_rate}")



Tasa de Conversión: 0.56
Ingresos Generados: 280893.33
Ratio de Retorno de Inversión (ROI): -0.859553335
Porcentaje de Retención de Clientes: 1.0
